### Выбрать наиближайшую к одному из офисов компании локацию для размещения рекламных баннеров.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# запрашиваем файл с данными и форматируем его
data = pd.read_csv('checkins.dat', sep='|', header=0, skipinitialspace=True)
data.dropna(inplace=True)

/home/thomas/anaconda2/envs/env_py2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
display(data.shape)
data.head(3)

(396634, 6)

,id,user_id,venue_id,latitude,longitude,created_at
2,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
4,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
8,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22


In [4]:
# удаляем неинформативные признаки
data = data.drop(columns=[u'id    ', u'user_id ', u'venue_id ', u'created_at      '])
data.head(3)

,latitude,longitude
2,38.895112,-77.036366
4,33.800745,-84.410520
8,45.523452,-122.676207


In [5]:
display(data.shape)
display(data.info())
# пропущенных значений нет

(396634, 2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396634 entries, 2 to 1021965
Data columns (total 2 columns):
latitude          396634 non-null float64
longitude         396634 non-null float64
dtypes: float64(2)
memory usage: 9.1 MB


None

In [6]:
# импортируем модуль с алгоритмом MeanShift
from sklearn.cluster import MeanShift

In [7]:
# создаем объект класса MeanShift с параметром bandwidth=0.1, что в переводе 
# из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.
# при этом по условию рекламодателю интересны кластера с не менее чем 15 точками
MS = MeanShift(bandwidth=0.1, min_bin_freq=15)
#clustering = MS.fit(data)

In [8]:
# прежде чем сократить выборку для обработки перемешиваю ее
from sklearn.utils import shuffle
data_shuffled = shuffle(data)
data_shuffled.head(3)

,latitude,longitude
119191,44.954167,-93.113889
140169,33.414768,-111.909309
572443,42.358431,-71.059773


In [9]:
# берем случайные 100 000 объектов из выборки. авторы задания утверждают, что это не
# приведет к потере качества алгоритма
data_short = data_shuffled.sample(n=100000)
display(data_short.shape)
data_short.head(3)

(100000, 2)

,latitude,longitude
895547,34.582770,-117.409215
542709,47.606209,-122.332071
422128,35.689487,139.691706


In [10]:
# обучаем алгоритм
clustering = MS.fit(data_short)

In [15]:
# возвращаем координаты центров кластеров(смотрим на первые для проверки) и их количество
display(clustering.cluster_centers_[0])
display(clustering.cluster_centers_.shape)

array([ 40.71786395, -73.98931849])

(3557, 2)

In [16]:
# создаем фрейм с координатами оффисов компании
d = {'latitude': [33.751277, 25.867736, 51.503016, 52.378894, 39.366487, -33.868457], 
     'longitude': [-118.188740, -80.324116, -0.075479, 4.885084, 117.036146, 151.205134], 
     'city': ['Los Angeles', 'Miami', 'London', 'Amsterdam', 'Beijing', 'Sydney']}
offices = pd.DataFrame(data=d)
offices

,city,latitude,longitude
0,Los Angeles,33.751277,-118.188740
1,Miami,25.867736,-80.324116
2,London,51.503016,-0.075479
3,Amsterdam,52.378894,4.885084
4,Beijing,39.366487,117.036146
5,Sydney,-33.868457,151.205134


In [41]:
from math import sqrt

In [49]:
# расчитываем расстояния от оффисов до центорв кластеров и находим наименьшее
min_dist = 1000000
city_num = 100
closest_latitude = 0
closest_longitude = 0

for j in range(clustering.cluster_centers_.shape[0]):
    
    for i in range(offices.shape[0]):
        dist = sqrt((offices['latitude'][i] - clustering.cluster_centers_[j][0])**2 + 
                    (offices['longitude'][i] - clustering.cluster_centers_[j][1])**2)
        if min_dist > dist:
            min_dist = dist
            city_num = i
            closest_latitude = clustering.cluster_centers_[j][0]
            closest_longitude =  clustering.cluster_centers_[j][1]

print min_dist, city_num
print closest_latitude, closest_longitude
        
        

0.00504029959668 5
-33.86342948448272 151.20477524137948


In [57]:
# сохраняем координаты центра кластера с наименьшим расстоянием до офиса компании заказчика
with open('carnival.txt', 'w') as f_out:
    f_out.write("{} {}".format(closest_latitude,closest_longitude))     